In [1]:
# read in files

# for each file, parse into soup

    # find all catchprases
        # for each catchprase, extract text and store in a list
    # save list as pickle to disk

    # find all sentences
        # for each sentence, extract text and store in a list
    # save list as pickle to disk

In [1]:
import re
import os
import nltk
import pickle
import justext
import nltk.data
import pandas as pd 
from nltk.tokenize import TreebankWordTokenizer
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize

# set directory to case reports
base_dir = 'data/fulltext'

# Make directory to store extracted catchphrases
catch_directory = os.path.dirname('data/catch_text/')
if not os.path.exists(catch_directory):
    os.makedirs(catch_directory)
    
# Make directory to store extracted sentences
sent_directory = os.path.dirname('data/sent_text/')
if not os.path.exists(sent_directory):
    os.makedirs(sent_directory)
    
# function for sentences
def gen_sentences(sentence):
    sents = []
    for sent in punkt_sentences:
        sents.append(sent)
    return sents
    
# read in case files
for num, filename in enumerate(os.scandir(base_dir)):
    with open(filename, 'rb') as case_file:
            # parse case file into beautiful soup
            soup = BeautifulSoup(case_file, 'xml')
            
            # obtain all catchphrases in the case file
            catchphrases = soup.find_all('catchphrase')
            
            # obtain all sentences in the case file
            sentences = soup.find_all('sentence')
            
            # obtain all text in identified catchphrase and store them together
            gold_catchphrases = []
            for catchphrase in catchphrases:
                text = catchphrase.text
                text = re.sub(r'.*>','',text)
                gold_catchphrases.append(text)
                
            # obtain all text in identified sentences and store them together
            gold_sentences = []
            for sentence in sentences:
                text = sentence.text
                text = re.sub('(\\d)', '', text)
                text = text.strip() 
                text = re.sub(r'.*>','',text)
                sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
                punkt_sentences = sentence_tokenizer.tokenize(text)
                text = gen_sentences(punkt_sentences)
                gold_sentences.append(text)
            
            num += 1
            num = str(num)
            
            #print(gold_sentences)
                        
            # save catchphrases as pickle
            pickle_out = open(os.path.join(catch_directory, num),"wb")
            pickle.dump(gold_catchphrases, pickle_out)
            pickle_out.close()
            
            # save sentences as pickle
            pickle_out = open(os.path.join(sent_directory, num),"wb")
            pickle.dump(gold_sentences, pickle_out)
            pickle_out.close()

In [72]:
#function for extracting tokens from catchphrases text and sentences text
def tokenextraction(catch_text, sent_text):
    catch_tokens = []
    sent_tokens = []
    for text in catch_text :
        catch_tokens += treebank_tokenizer.tokenize(text)
    for l in sent_text :
        for text in l :
            sent_tokens += treebank_tokenizer.tokenize(text)
    return catch_tokens, sent_tokens

In [73]:
#iterating through all the files for extracting tokens
treebank_tokenizer = TreebankWordTokenizer()

all_catch_tokens = []
all_sent_tokens = []
for i in range(1,3890):
    try :
        catch_text = pd.read_pickle("data/catch_text/"+str(i))
        sent_text = pd.read_pickle("data/sent_text/"+str(i))
    except EOFError:
        catch_text = list()
    catch_tokens, sent_tokens = tokenextraction(catch_text, sent_text)
    all_catch_tokens.extend(catch_tokens)
    all_sent_tokens.extend(sent_tokens)


In [ ]:
print(all_catch_tokens)
# print(all_sent_tokens)

In [ ]:
# Cleaning the tokens function
def cleanedtokens(tokens):
    cleaned_tokens = [w for w in tokens if w.isalnum()]
    cleaned_tokens = [w for w in tokens if not w in stop_words]

In [77]:
# Cleaning the tokens
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

all_catch_cleaned_tokens = cleanedtokens(all_catch_tokens)
print(all_catch_cleaned_tokens)
all_sent_cleaned_tokens = cleanedtokens(all_sent_tokens)
# print(all_sent_cleaned_tokens)

['indemnity', 'costs', 'calderbank', 'letter', 'refusal', 'offer', 'whether', 'refusal', 'unreasonable', 'variety', 'factors', 'relevant', 'assessment', 'unreasonableness', 'indemnity', 'costs', 'refused', 'costs', 'judicial', 'review', 'application', 'brought', 'pursuant', 'administrative', 'decisions', 'judicial', 'review', 'act', '1977', 'adjr', 'act', 'review', 'decision', 'respondent', 'objection', 'competency', 'application', 'whether', 'decision', 'open', 'review', 'meaning', 'adjr', 'act', 'whether', 'substantive', 'determination', 'whether', 'applicant', 'standing', 'bring', 'application', 'administrative', 'law', 'interlocutory', 'injunction', 'quia', 'timet', 'injunction', 'whether', 'serious', 'question', 'tried', 'whether', 'applicant', 'likely', 'suffer', 'injury', 'damages', 'adequate', 'remedy', 'whether', 'balance', 'convenience', 'favours', 'granting', 'injunction', 'alleged', 'contravention', '52', 'trade', 'practices', 'act', '1974', 'cth', 'practice', 'procedure', 